In [43]:
import pandas as pd
import numpy as np

In [44]:
#Last Trading Day
# Trading will cease at the close of business two UK Business Days prior to
# the first calendar day of the delivery month, quarter, season, or calendar.

ice_ttf= pd.read_excel('curves/data/ICE_TTF_FUT.xlsx')  
ice_ttf.columns = ["tradedate"] + [f'M{i:02}' for i in range(1, 61)]
ice_ttf['tradedate'] = pd.to_datetime(ice_ttf['tradedate'])

In [45]:
# Reset the index to make 'trade_date' a column again for melting
ice_ttf_long = pd.melt(ice_ttf, id_vars=['tradedate'], var_name='tenor', value_name='price')

In [53]:
# Convert 'month_number' to string if it's not already, then extract the last two characters and convert to integer
ice_ttf_long['tenor_offset'] = ice_ttf_long['tenor'].astype(str).str[-2:].astype(int)
ice_ttf_long

,tradedate,tenor,price,offset,offset_adjustment,tenor_offset
0,2013-01-21,M01,26.040,1,0,1
1,2013-01-22,M01,26.340,1,0,1
2,2013-01-23,M01,26.330,1,0,1
3,2013-01-24,M01,25.850,1,0,1
4,2013-01-25,M01,26.100,1,0,1
...,...,...,...,...,...,...
160495,2023-05-24,M60,25.887,60,0,60
160496,2023-05-25,M60,24.801,60,0,60
160497,2023-05-26,M60,25.061,60,0,60
160498,2023-05-29,M60,25.036,60,0,60


In [54]:
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import AbstractHolidayCalendar, DateOffset, Holiday, next_monday, next_monday_or_tuesday, GoodFriday, EasterMonday, nearest_workday, MO

# Define a custom UK business day calendar that accounts for UK public holidays
class UKBusinessCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Year’s Day', month=1, day=1, observance=nearest_workday),
        GoodFriday,
        EasterMonday,
        Holiday('Early May Bank Holiday', month=5, day=1, offset=DateOffset(weekday=MO(1))),
        Holiday('Spring Bank Holiday', month=5, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Summer Bank Holiday', month=8, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Christmas Day', month=12, day=25, observance=next_monday),
        Holiday('Boxing Day', month=12, day=26, observance=next_monday_or_tuesday)
    ]
# Create a custom business day offset using the UK business calendar
uk_bd = CustomBusinessDay(calendar=UKBusinessCalendar())

# Function to calculate the first trading day which is one UK business days before the first of next month
# Last trading day is two UK business days before the first of next month
def calculate_first_trading_day(date):
    first_day_next_month = pd.Timestamp(date.year + int(date.month == 12), date.month % 12 + 1, 1)
    first_trading_day = first_day_next_month - 1 * uk_bd
    return first_trading_day

In [ ]:
df_melted

,tradedate,tenor,price
0,2013-01-21,M01,26.040
1,2013-01-22,M01,26.340
2,2013-01-23,M01,26.330
3,2013-01-24,M01,25.850
4,2013-01-25,M01,26.100
...,...,...,...
160495,2023-05-24,M60,25.887
160496,2023-05-25,M60,24.801
160497,2023-05-26,M60,25.061
160498,2023-05-29,M60,25.036


In [ ]:
prices = raw

logret = np.log(prices.shift(1)/prices)

NameError: name 'raw' is not defined